<a href="https://colab.research.google.com/github/joaogcoward/n8n-marine-conservation/blob/main/WebSocket_connection_to_aisstreamio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install websockets requests

In [15]:
import asyncio
import websockets
import json
import requests
import time

# --- Your Configuration ---
YOUR_API_KEY = "a8b65e26ed61485dc6828fc3f53021d9711cac37"
N8N_WEBHOOK_URL = "https://joagc.app.n8n.cloud/webhook/85ffeef7-c116-4c02-8be8-a8f4748debbb"

# Bounding Boxes for Raja Ampat and surrounding shipping lanes
# Each bounding box is in the format [[min_lat, min_lon], [max_lat, max_lon]]
# We will use an array to subscribe to all of them at once.
BOUNDING_BOXES = [
    # 1. Main Raja Ampat Conservation Area
    [[-3.0, 129.0], [0.5, 132.0]],
    # 2. Shipping lane near Sorong, Indonesia (major port)
    [[-1.0, 131.0], [0.0, 132.0]],
    # 3. Wider shipping channel approaching the region
    [[-2.0, 128.0], [1.0, 130.0]]
]

async def listen_to_ais():
    uri = "wss://stream.aisstream.io/v0/stream"

    # This is the corrected JSON payload based on the official GitHub issue
    subscribe_message = {
        "APIKey": YOUR_API_KEY,
        "BoundingBoxes": BOUNDING_BOXES
    }

    while True:
        try:
            async with websockets.connect(uri) as websocket:
                await websocket.send(json.dumps(subscribe_message))
                print("Subscribed to AIS stream. Listening for messages...")

                while True:
                    try:
                        message = await asyncio.wait_for(websocket.recv(), timeout=30)
                        data = json.loads(message)

                        if 'ErrorMessage' not in data and 'MessageType' in data:
                            requests.post(N8N_WEBHOOK_URL, json=data)
                            print("Message forwarded to n8n.")
                        else:
                            print("Received non-vessel message (or error):", data)

                    except asyncio.TimeoutError:
                        print("No message received in 30 seconds. Pinging to keep connection alive...")
                        await websocket.ping()
                    except json.JSONDecodeError:
                        print("Received non-JSON message.")
                    except websockets.exceptions.ConnectionClosed:
                        print("Connection closed by server.")
                        break
        except Exception as e:
            print(f"Connection failed or interrupted: {e}")
            print("Will attempt to reconnect in 10 seconds.")
            time.sleep(10)

await listen_to_ais()

Subscribed to AIS stream. Listening for messages...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30 seconds. Pinging to keep connection alive...
No message received in 30

CancelledError: 